# Pycaret을 이용한 베이스라인 모델 (+ 제공된 외부 데이터 활용)

# [0] 모듈 로드

- 서드파트 모듈 설치

In [ ]:
!pip install -q pycaret
!pip install --upgrade -q xgboost
!pip install -q catboost
! pip install holidays

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.7/484.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.0 MB/s et

- 라이브러리 로드

In [ ]:
import pandas as pd
import numpy as np
import holidays


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

import warnings

warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# [1] 데이터 로드

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/test.csv', encoding = 'utf-8')
sub = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/sample_submission.csv', encoding = 'utf-8')
countrywide = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/countrywide_accident.csv', encoding = 'utf-8')
cctv = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/대구 CCTV 정보.csv', encoding = 'cp949')
park = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/대구 주차장 정보.csv', encoding = 'cp949')
child = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/대구 어린이 보호 구역 정보.csv', encoding = 'cp949')
security = pd.read_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/data/대구 보안등 정보.csv', encoding = 'cp949')

In [ ]:
train.columns

Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형',
       '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도',
       '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수',
       '경상자수', '부상자수', 'ECLO'],
      dtype='object')

In [ ]:
train.head()

ID           사고일시   요일 기상상태            시군구      도로형태 노면상태  \
0  ACCIDENT_00000  2019-01-01 00  화요일   맑음   대구광역시 중구 대신동  단일로 - 기타   건조   
1  ACCIDENT_00001  2019-01-01 00  화요일   흐림  대구광역시 달서구 감삼동  단일로 - 기타   건조   
2  ACCIDENT_00002  2019-01-01 01  화요일   맑음  대구광역시 수성구 두산동  단일로 - 기타   건조   
3  ACCIDENT_00003  2019-01-01 02  화요일   맑음   대구광역시 북구 복현동  단일로 - 기타   건조   
4  ACCIDENT_00004  2019-01-01 04  화요일   맑음   대구광역시 동구 신암동  단일로 - 기타   건조   

   사고유형 사고유형 - 세부분류     법규위반 가해운전자 차종 가해운전자 성별 가해운전자 연령 가해운전자 상해정도 피해운전자 차종  \
0  차대사람  길가장자리구역통행중  안전운전불이행       승용        여      51세       상해없음      보행자   
1  차대사람       보도통행중       기타       승용        남      39세       상해없음      보행자   
2  차대사람       차도통행중  안전운전불이행       승용        남      70세       상해없음      보행자   
3   차대차          추돌  안전운전불이행       승용        남      49세       상해없음       승용   
4   차대차          추돌  안전운전불이행       승용        남      30세       상해없음       승용   

  피해운전자 성별 피해운전자 연령 피해운전자 상해정도  사망자수  중상자수  경상자수  부상자수  ECLO  
0        여      70세         중상     0     1     0     0     5  
1        남      61세         경상     0     0     1     0     3  
2        남      38세         경상     0     0     1     0     3  
3        남      36세         중상     0     1     0     0     5  
4        남      52세         경상     0     0     1     0     3

In [ ]:
test.head()

ID           사고일시   요일 기상상태              시군구        도로형태 노면상태  \
0  ACCIDENT_39609  2022-01-01 01  토요일   맑음     대구광역시 수성구 상동  교차로 - 교차로안   건조   
1  ACCIDENT_39610  2022-01-01 01  토요일   맑음    대구광역시 수성구 지산동    단일로 - 기타   건조   
2  ACCIDENT_39611  2022-01-01 04  토요일   맑음  대구광역시 수성구 수성동2가  교차로 - 교차로안   건조   
3  ACCIDENT_39612  2022-01-01 04  토요일   맑음    대구광역시 수성구 신매동    단일로 - 기타   건조   
4  ACCIDENT_39613  2022-01-01 06  토요일   맑음    대구광역시 달서구 감삼동  교차로 - 교차로안   건조   

   사고유형  
0  차대사람  
1  차대사람  
2   차대차  
3   차대차  
4   차대차

# [2] 피쳐 엔지니어링

In [ ]:
list(test.columns)

['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형']

In [ ]:
data = train[list(test.columns)]
data_y = train['ECLO']

## 2-1 피쳐 세분화

- 시군구 세분화

In [ ]:
split_sgg = data['시군구'].str.split(' ', expand=True)
data['구'] = split_sgg[1]
data['동'] = split_sgg[2]
data.drop(['시군구'], inplace = True, axis = 1)

In [ ]:
split_sgg = test['시군구'].str.split(' ', expand=True)
test['구'] = split_sgg[1]
test['동'] = split_sgg[2]
test.drop(['시군구'], inplace = True, axis = 1)

In [ ]:
data.head(2)

ID           사고일시   요일 기상상태      도로형태 노면상태  사고유형    구    동
0  ACCIDENT_00000  2019-01-01 00  화요일   맑음  단일로 - 기타   건조  차대사람   중구  대신동
1  ACCIDENT_00001  2019-01-01 00  화요일   흐림  단일로 - 기타   건조  차대사람  달서구  감삼동

- 사고일시세분화

In [ ]:
split_time = data['사고일시'].str.split('-', expand=True)
tmp_time = split_time[2].str.split(' ', expand = True)
data['연'] = split_time[0].astype(int)
data['월'] = split_time[1].astype(int)
data['일'] = tmp_time[0].astype(int)
data['시간'] = tmp_time[1].astype(int)

#data.drop(['사고일시'], inplace = True, axis = 1)

In [ ]:
split_time = test['사고일시'].str.split('-', expand=True)
tmp_time = split_time[2].str.split(' ', expand = True)
test['연'] = split_time[0].astype(int)
test['월'] = split_time[1].astype(int)
test['일'] = tmp_time[0].astype(int)
test['시간'] = tmp_time[1].astype(int)

#test.drop(['사고일시'], inplace = True, axis = 1)

In [ ]:
data.head(2)

ID           사고일시   요일 기상상태      도로형태 노면상태  사고유형    구    동  \
0  ACCIDENT_00000  2019-01-01 00  화요일   맑음  단일로 - 기타   건조  차대사람   중구  대신동   
1  ACCIDENT_00001  2019-01-01 00  화요일   흐림  단일로 - 기타   건조  차대사람  달서구  감삼동   

      연  월  일  시간  
0  2019  1  1   0  
1  2019  1  1   0

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      39609 non-null  object
 1   사고일시    39609 non-null  object
 2   요일      39609 non-null  object
 3   기상상태    39609 non-null  object
 4   도로형태    39609 non-null  object
 5   노면상태    39609 non-null  object
 6   사고유형    39609 non-null  object
 7   구       39609 non-null  object
 8   동       39609 non-null  object
 9   연       39609 non-null  int64 
 10  월       39609 non-null  int64 
 11  일       39609 non-null  int64 
 12  시간      39609 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 3.9+ MB


- 계절 추가

In [ ]:
data['일시'] = pd.to_datetime(data['사고일시'].str[0:10])
test['일시'] = pd.to_datetime(test['사고일시'].str[0:10])

In [ ]:
def to_season( x ):
    if x in [12, 1, 2]:
        return 0 # 겨울
    elif x in [3, 4, 5]:
        return 1 # 봄
    elif x in [6, 7, 8]:
        return 2 # 여름
    else:
        return 3 # 가을
    pass

In [ ]:
data['season'] = data['월'].apply(lambda x : to_season(x))
test['season'] = test['월'].apply(lambda x : to_season(x))

- 분기 추가

In [ ]:
def to_quarter( x ):
    if x in [1,2,3]:
        return 0 # 1분기
    elif x in [4,5,6]:
        return 1 # 2분기
    elif x in [7, 8, 9]:
        return 2 # 3분기
    else:
        return 3 # 4분기
    pass


In [ ]:
data['분기'] = data['월'].apply(lambda x : to_quarter(x))
test['분기'] = test['월'].apply(lambda x : to_quarter(x))

- 휴일 추가

In [ ]:
# 한국 휴일 객체 생성
kr_holidays = holidays.KR()

# generate holiday table
holiday_df = pd.DataFrame(columns=['ds','holiday'])
holiday_df['ds'] = sorted(data['일시'])
holiday_df['holiday'] = holiday_df.ds.apply(lambda x: 1 if x in kr_holidays else 0)
data['holiday'] = holiday_df['holiday']

In [ ]:
holiday_df = pd.DataFrame(columns=['ds','holiday'])
holiday_df['ds'] = sorted(test['일시'])
holiday_df['holiday'] = holiday_df.ds.apply(lambda x: 1 if x in kr_holidays else 0)
test['holiday'] = holiday_df['holiday']

In [ ]:
data.head(2)

ID           사고일시   요일 기상상태      도로형태 노면상태  사고유형    구    동  \
0  ACCIDENT_00000  2019-01-01 00  화요일   맑음  단일로 - 기타   건조  차대사람   중구  대신동   
1  ACCIDENT_00001  2019-01-01 00  화요일   흐림  단일로 - 기타   건조  차대사람  달서구  감삼동   

      연  월  일  시간         일시  season  분기  holiday  
0  2019  1  1   0 2019-01-01       0   0        1  
1  2019  1  1   0 2019-01-01       0   0        1

In [ ]:
test.head(2)

ID           사고일시   요일 기상상태        도로형태 노면상태  사고유형    구    동  \
0  ACCIDENT_39609  2022-01-01 01  토요일   맑음  교차로 - 교차로안   건조  차대사람  수성구   상동   
1  ACCIDENT_39610  2022-01-01 01  토요일   맑음    단일로 - 기타   건조  차대사람  수성구  지산동   

      연  월  일  시간         일시  season  분기  holiday  
0  2022  1  1   1 2022-01-01       0   0        1  
1  2022  1  1   1 2022-01-01       0   0        1

# 2-2. 외부데이터 병합

- 전국 데이터

In [ ]:
countrywide.head(2)
# 전국 데이터, 일별, 요일별, 월별, 계절별, 요일별 다른 특징으로 접근해볼 수 있겠다.

ID           사고일시   요일 기상상태            시군구  \
0  COUNTRYWIDE_ACCIDENT_000000  2019-01-01 00  화요일   맑음  서울특별시 강서구 방화동   
1  COUNTRYWIDE_ACCIDENT_000001  2019-01-01 00  화요일   맑음    경기도 포천시 소흘읍   

             도로형태 노면상태  사고유형 사고유형 - 세부분류       법규위반 가해운전자 차종 가해운전자 성별  \
0  교차로 - 교차로횡단보도내   건조  차대사람         횡단중  보행자보호의무위반       승용        남   
1      교차로 - 교차로안   건조   차대차          기타  직진우회전진행방해       승용        남   

  가해운전자 연령 가해운전자 상해정도 피해운전자 차종 피해운전자 성별 피해운전자 연령 피해운전자 상해정도  사망자수  중상자수  경상자수  \
0      26세       상해없음      보행자        남      40세         경상     0     0     1   
1      54세       상해없음       이륜        남      20세         경상     0     0     1   

   부상자수  ECLO  
0     0     3  
1     0     3

### CCTV 데이터

In [ ]:
cctv.head(2)

무인교통단속카메라관리번호    시도명 시군구명 도로종류 도로노선번호 도로노선명  도로노선방향          소재지도로명주소  \
0          주차12  대구광역시   남구   시도    NaN   대명로       3  대구광역시 남구 대명로 162   
1          주차13  대구광역시   남구   시도    NaN   봉덕로       3  대구광역시 남구 봉덕로 135   

               소재지지번주소         위도          경도           설치장소  단속구분  제한속도  \
0  대구광역시 남구 대명동 903-13  35.838857  128.575418  안지랑네거리1 버스정류장     4     0   
1   대구광역시 남구 봉덕동 853-8  35.844653  128.604619         효성백년가약     4     0   

   단속구간위치구분  과속단속구간길이  보호구역구분    설치연도  
0       NaN       NaN    99.0  2010.0  
1       NaN       NaN    99.0  2013.0

In [ ]:
a = list(data['동'].unique())

In [ ]:
b = list(cctv['소재지지번주소'].str.split(' ', expand=True)[2].unique())

In [ ]:
len([value for value in a if value not in b]) # 50개의 다른 동이 존재한다.

50

In [ ]:
cctv['동'] = cctv['소재지지번주소'].str.split(' ', expand=True)[2]

In [ ]:
cctv_gu = cctv[['시군구명','위도','경도','제한속도']]
cctv_dong = cctv[['동','위도','경도','제한속도']]

- 구단위 그룹화

In [ ]:
cctv_gu_tmp = cctv_gu.groupby(['시군구명']).agg({'위도' : 'std', '경도' : 'std', '제한속도' : 'mean'}).reset_index().rename(columns = {'위도' : 'cctv_위도_std_gu', '경도' : 'cctv_경도_std_gu', '제한속도' : 'cctv_제한속도_평균_gu'})
cctv_gu_tmp1 = cctv_gu.groupby(['시군구명']).agg({'위도' : 'count'}).reset_index().rename(columns = {'위도' : 'cctv_count_gu'})
cctv_gu_tmp['cctv_count_gu'] = cctv_gu_tmp1['cctv_count_gu']

- 동단위 그룹화

In [ ]:
cctv_dong_tmp = cctv_dong.groupby(['동']).agg({'위도' : 'std', '경도' : 'std', '제한속도' : 'mean'}).reset_index().rename(columns = {'위도' : 'cctv_위도_std_dong', '경도' : 'cctv_경도_std_dong', '제한속도' : 'cctv_제한속도_평균_dong'})
cctv_dong_tmp1 = cctv_dong.groupby(['동']).agg({'위도' : 'count'}).reset_index().rename(columns = {'위도' : 'cctv_count_dong'})
cctv_dong_tmp['cctv_count_dong'] = cctv_dong_tmp1['cctv_count_dong']

In [ ]:
data = data.merge(cctv_dong_tmp, how = 'left', on = '동').merge(cctv_gu_tmp, how = 'left', left_on = '구', right_on = '시군구명')
test = test.merge(cctv_dong_tmp, how = 'left', on = '동').merge(cctv_gu_tmp, how = 'left', left_on = '구', right_on = '시군구명')

In [ ]:
data = data.where(pd.notnull(data), data.min(), axis='columns')
test = test.where(pd.notnull(test), test.min(), axis='columns')

### 주차장 데이터
    * 동별
        - 주차구획수 합, 위도 std, 경도 std
    * 구별
        - 주차구획수 합, 위도 std, 경도 std

In [ ]:
park.columns, park.shape

(Index(['주차장관리번호', '주차장명', '주차장구분', '주차장유형', '소재지도로명주소', '소재지지번주소', '주차구획수',
        '급지구분', '부제시행구분', '운영요일', '평일운영시작시각', '평일운영종료시각', '토요일운영시작시각',
        '토요일운영종료시각', '공휴일운영시작시각', '공휴일운영종료시각', '요금정보', '주차기본시간', '주차기본요금',
        '추가단위시간', '추가단위요금', '1일주차권요금적용시간', '1일주차권요금', '월정기권요금', '결제방법', '특기사항',
        '위도', '경도', '데이터기준일자'],
       dtype='object'),
 (871, 29))

In [ ]:
park['시군구명'] = park['소재지지번주소'].str.split(' ', expand=True)[1]
park['동'] = park['소재지지번주소'].str.split(' ', expand=True)[2]

In [ ]:
park_gu = park[['시군구명','위도','경도','주차구획수']]
park_dong = park[['동','위도','경도','주차구획수']]

- 구단위 그룹화

In [ ]:
park_gu_tmp = park_gu.groupby(['시군구명']).agg({'위도' : 'std', '경도' : 'std', '주차구획수' : 'sum'}).reset_index().rename(columns = {'위도' : 'park_위도_std_gu', '경도' : 'park_경도_std_gu', '주차구획수' : '주차구획수_gu'})

- 동단위 그룹화

In [ ]:
park_dong_tmp = park_dong.groupby(['동']).agg({'위도' : 'std', '경도' : 'std', '주차구획수' : 'sum'}).reset_index().rename(columns = {'위도' : 'park_위도_std_dong', '경도' : 'park_경도_std_dong','주차구획수' : '주차구획수_dong'})

- 원본과 결합

In [ ]:
data = data.merge(park_dong_tmp, how = 'left', on = '동').merge(park_gu_tmp, how = 'left', left_on = '구', right_on = '시군구명')
test = test.merge(park_dong_tmp, how = 'left', on = '동').merge(park_gu_tmp, how = 'left', left_on = '구', right_on = '시군구명')

In [ ]:
data = data.where(pd.notnull(data), data.min(), axis='columns')
test = test.where(pd.notnull(test), test.min(), axis='columns')

### 어린이보호구역 데이터

In [ ]:
child.head(2)

시설종류   대상시설명               소재지도로명주소            소재지지번주소         위도  \
0  초등학교  남도초등학교  대구광역시 남구 현충동길 74(대명동)  대구광역시 남구 대명동 1709  35.845027   
1  초등학교  영선초등학교    대구광역시 남구 영선길96(이천동)   대구광역시 남구 이천동 477  35.852901   

           경도  관리기관명 관할경찰서명 CCTV설치여부  CCTV설치대수 보호구역도로폭     데이터기준일자  
0  128.581402  대구광역시  남부경찰서        Y       7.0     6~8  2020-03-23  
1  128.596014  대구광역시  남부경찰서        Y       8.0    6~10  2020-03-23

In [ ]:
child.shape

(640, 12)

In [ ]:
child['시군구명'] = child['소재지지번주소'].str.split(' ', expand=True)[1]
child['동'] = child['소재지지번주소'].str.split(' ', expand=True)[2]

In [ ]:
child_gu = child[['시군구명','위도','경도','CCTV설치대수','대상시설명']]
child_dong = child[['동','위도','경도','CCTV설치대수','대상시설명']]

- 구단위 그룹화

In [ ]:
child_gu_tmp = child_gu.groupby(['시군구명']).agg({'위도' : 'std', '경도' : 'std', 'CCTV설치대수' : 'sum', '대상시설명':'count'}).reset_index().rename(columns = {'위도' : 'child_위도_std_gu', '경도' : 'child_경도_std_gu', 'CCTV설치대수' : 'CCTV설치대수_gu', '대상시설명' : 'child_count_gu'})

- 동단위 그룹화

In [ ]:
child_dong_tmp = child_dong.groupby(['동']).agg({'위도' : 'std', '경도' : 'std', 'CCTV설치대수' : 'sum', '대상시설명':'count'}).reset_index().rename(columns = {'위도' : 'child_위도_std_dong', '경도' : 'child_경도_std_dong', 'CCTV설치대수' : 'CCTV설치대수_dong', '대상시설명' : 'child_count_dong'})

- 원본과 결합

In [ ]:
data = data.merge(child_dong_tmp, how = 'left', on = '동').merge(child_gu_tmp, how = 'left', left_on = '구', right_on = '시군구명')
test = test.merge(child_dong_tmp, how = 'left', on = '동').merge(child_gu_tmp, how = 'left', left_on = '구', right_on = '시군구명')

In [ ]:
data = data.where(pd.notnull(data), data.min(), axis='columns')
test = test.where(pd.notnull(test), test.min(), axis='columns')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39609 entries, 0 to 39608
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ID                 39609 non-null  object        
 1   사고일시               39609 non-null  object        
 2   요일                 39609 non-null  object        
 3   기상상태               39609 non-null  object        
 4   도로형태               39609 non-null  object        
 5   노면상태               39609 non-null  object        
 6   사고유형               39609 non-null  object        
 7   구                  39609 non-null  object        
 8   동                  39609 non-null  object        
 9   연                  39609 non-null  int64         
 10  월                  39609 non-null  int64         
 11  일                  39609 non-null  int64         
 12  시간                 39609 non-null  int64         
 13  일시                 39609 non-null  datetime64[ns]
 14  season

- 보안등데이터

In [ ]:
security.shape

(71913, 8)

In [ ]:
security.head(2)

보안등위치명  설치개수              소재지도로명주소               소재지지번주소         위도  \
0  대명1동1     1      대구광역시 남구 현충로 155   대구광역시 남구 대명동 1722-4  35.846703   
1  대명1동2     1  대구광역시 남구 현충로31길 10-1  대구광역시 남구 대명동 1723-11  35.846863   

           경도    설치연도 설치형태  
0  128.579803  2016.0  한전주  
1  128.579406  2016.0  한전주

In [ ]:
security['시군구명'] = security['소재지지번주소'].str.split(' ', expand=True)[1]
security['동'] = security['소재지지번주소'].str.split(' ', expand=True)[2]

In [ ]:
security_gu = security[['시군구명','위도','경도','설치개수','보안등위치명']]
security_dong = security[['동','위도','경도','설치개수','보안등위치명']]

- 구단위 그룹화

In [ ]:
security_gu_tmp = security_gu.groupby(['시군구명']).agg({'위도' : 'std', '경도' : 'std', '설치개수' : 'sum'}).reset_index().rename(columns = {'위도' : 'security_위도_std_gu', '경도' : 'security_경도_std_gu', '설치개수' : 'security설치대수_gu', '보안등위치명' : 'security_count_gu'})

- 동단위 그룹화

In [ ]:
security_dong_tmp = security_dong.groupby(['동']).agg({'위도' : 'std', '경도' : 'std', '설치개수' : 'sum'}).reset_index().rename(columns = {'위도' : 'security_위도_std_dong', '경도' : 'security_경도_std_dong', '설치개수' : 'security설치대수_dong'})

In [ ]:
security_dong_tmp

동  security_위도_std_dong  security_경도_std_dong  security설치대수_dong
0       988-130                   NaN                   NaN                  1
1           가창면                   NaN                   NaN               1123
2           각산동              0.002685              0.004169                139
3           갈산동              0.003868              0.004576                349
4    갈산동358-171                   NaN                   NaN                  1
..          ...                   ...                   ...                ...
974         화원읍                   NaN                   NaN                940
975         화전동              0.000367              0.000277                 22
976        황금1동                   NaN                   NaN                153
977        황금2동                   NaN                   NaN                590
978         효목동              0.003603              0.004426               1164

[979 rows x 4 columns]

- 원본과 결합

In [ ]:
data = data.merge(security_dong_tmp, how = 'left', on = '동').merge(security_gu_tmp, how = 'left', left_on = '구', right_on = '시군구명')
test = test.merge(security_dong_tmp, how = 'left', on = '동').merge(security_gu_tmp, how = 'left', left_on = '구', right_on = '시군구명')

In [ ]:
data.drop(['시군구명_x','시군구명_y'], axis = 1, inplace = True)
test.drop(['시군구명_x','시군구명_y'], axis = 1, inplace = True)

In [ ]:
data = data.where(pd.notnull(data), data.mean(), axis='columns')
test = test.where(pd.notnull(test), test.mean(), axis='columns')

## 데이터 확인

In [ ]:
data.columns

Index(['ID', '사고일시', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '구', '동', '연', '월',
       '일', '시간', '일시', 'season', '분기', 'holiday', 'cctv_위도_std_dong',
       'cctv_경도_std_dong', 'cctv_제한속도_평균_dong', 'cctv_count_dong',
       'cctv_위도_std_gu', 'cctv_경도_std_gu', 'cctv_제한속도_평균_gu', 'cctv_count_gu',
       'park_위도_std_dong', 'park_경도_std_dong', '주차구획수_dong', 'park_위도_std_gu',
       'park_경도_std_gu', '주차구획수_gu', 'child_위도_std_dong', 'child_경도_std_dong',
       'CCTV설치대수_dong', 'child_count_dong', 'child_위도_std_gu',
       'child_경도_std_gu', 'CCTV설치대수_gu', 'child_count_gu',
       'security_위도_std_dong', 'security_경도_std_dong', 'security설치대수_dong',
       'security_위도_std_gu', 'security_경도_std_gu', 'security설치대수_gu'],
      dtype='object')

In [ ]:
test

ID           사고일시   요일 기상상태         도로형태 노면상태  사고유형    구  \
0      ACCIDENT_39609  2022-01-01 01  토요일   맑음   교차로 - 교차로안   건조  차대사람  수성구   
1      ACCIDENT_39610  2022-01-01 01  토요일   맑음     단일로 - 기타   건조  차대사람  수성구   
2      ACCIDENT_39611  2022-01-01 04  토요일   맑음   교차로 - 교차로안   건조   차대차  수성구   
3      ACCIDENT_39612  2022-01-01 04  토요일   맑음     단일로 - 기타   건조   차대차  수성구   
4      ACCIDENT_39613  2022-01-01 06  토요일   맑음   교차로 - 교차로안   건조   차대차  달서구   
...               ...            ...  ...  ...          ...  ...   ...  ...   
10958  ACCIDENT_50567  2022-12-31 18  토요일   맑음     단일로 - 터널   건조   차대차   남구   
10959  ACCIDENT_50568  2022-12-31 18  토요일   맑음     단일로 - 기타   건조   차대차  수성구   
10960  ACCIDENT_50569  2022-12-31 20  토요일   맑음     단일로 - 기타   건조   차대차  수성구   
10961  ACCIDENT_50570  2022-12-31 20  토요일   맑음  교차로 - 교차로부근   건조   차대차  수성구   
10962  ACCIDENT_50571  2022-12-31 21  토요일   맑음  교차로 - 교차로부근   건조   차대차   동구   

           동     연   월   일  시간         일시  season  분기  holiday  \
0         상동  2022   1   1   1 2022-01-01       0   0        1   
1        지산동  2022   1   1   1 2022-01-01       0   0        1   
2      수성동2가  2022   1   1   4 2022-01-01       0   0        1   
3        신매동  2022   1   1   4 2022-01-01       0   0        1   
4        감삼동  2022   1   1   6 2022-01-01       0   0        1   
...      ...   ...  ..  ..  ..        ...     ...  ..      ...   
10958    대명동  2022  12  31  18 2022-12-31       0   3        0   
10959    시지동  2022  12  31  18 2022-12-31       0   3        0   
10960    연호동  2022  12  31  20 2022-12-31       0   3        0   
10961    범물동  2022  12  31  20 2022-12-31       0   3        0   
10962    효목동  2022  12  31  21 2022-12-31       0   3        0   

      cctv_위도_std_dong cctv_경도_std_dong cctv_제한속도_평균_dong  cctv_count_dong  \
0             0.005473         0.003891              52.5                4   
1             0.003632         0.006233         47.142857                7   
2                  0.0              0.0              40.0                1   
3              0.00303         0.004689         42.727273               11   
4             0.003379         0.004823              17.5               12   
...                ...              ...               ...              ...   
10958          0.00829         0.010965         20.483871               62   
10959         0.002854         0.004911              52.5                4   
10960              0.0              0.0               0.0                1   
10961         0.002105         0.003478         44.285714                7   
10962         0.004158         0.002067              42.5                4   

       cctv_위도_std_gu  cctv_경도_std_gu  cctv_제한속도_평균_gu  cctv_count_gu  \
0            0.017701        0.032648        50.784314            102   
1            0.017701        0.032648        50.784314            102   
2            0.017701        0.032648        50.784314            102   
3            0.017701        0.032648        50.784314            102   
4            0.018203        0.021047        23.862816            277   
...               ...             ...              ...            ...   
10958        0.007748        0.014683        18.936170             94   
10959        0.017701        0.032648        50.784314            102   
10960        0.017701        0.032648        50.784314            102   
10961        0.017701        0.032648        50.784314            102   
10962        0.036155        0.039577        55.434783             92   

      park_위도_std_dong park_경도_std_dong  주차구획수_dong park_위도_std_gu  \
0             0.000045         0.000063          10       0.014076   
1             0.000106         0.005575         183       0.014076   
2             0.000045         0.000063          10       0.014076   
3             0.001476         0.003329         153       0.014076   
4             0.003383         0.000738         114       0.015224   
...                ...              ...    

# [3] 인코딩

## 2-1. 라벨 인코딩

In [ ]:
qual_col = ['요일', '기상상태', '도로형태', '노면상태', '사고유형', '구', '동']

for i in qual_col:
    le = LabelEncoder()
    data[i]=le.fit_transform(data[i])
    test[i]=le.transform(test[i])

print('Done.')

Done.


In [ ]:
data.drop(['ID'], axis = 1, inplace = True)
test.drop(['ID'], axis = 1, inplace = True)

In [ ]:
data.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [ ]:
data

사고일시  요일  기상상태  도로형태  노면상태  사고유형  구    동     연   월   일  시간  \
0      2019-01-01 00   6     2     6     0     0  7   40  2019   1   1   0   
1      2019-01-01 00   6     5     6     0     0  1    4  2019   1   1   0   
2      2019-01-01 01   6     2     6     0     0  6   66  2019   1   1   1   
3      2019-01-01 02   6     2     6     0     1  4   79  2019   1   1   2   
4      2019-01-01 04   6     2     6     0     1  3  129  2019   1   1   4   
...              ...  ..   ...   ...   ...   ... ..  ...   ...  ..  ..  ..   
39604  2021-12-31 19   0     2     1     0     1  6  118  2021  12  31  19   
39605  2021-12-31 19   0     2     6     0     1  1  103  2021  12  31  19   
39606  2021-12-31 21   0     2     1     0     1  1  144  2021  12  31  21   
39607  2021-12-31 22   0     2     3     0     1  1  158  2021  12  31  22   
39608  2021-12-31 23   0     2     7     0     1  5   89  2021  12  31  23   

              일시  season  분기  holiday cctv_위도_std_dong cctv_경도_std_dong  \
0     2019-01-01       0   0        1         0.001808         0.001888   
1     2019-01-01       0   0        1         0.003379         0.004823   
2     2019-01-01       0   0        1         0.003966         0.002731   
3     2019-01-01       0   0        1         0.003797         0.003026   
4     2019-01-01       0   0        1         0.004666          0.00602   
...          ...     ...  ..      ...              ...              ...   
39604 2021-12-31       0   3        0              0.0              0.0   
39605 2021-12-31       0   3        0         0.005133         0.009326   
39606 2021-12-31       0   3        0          0.00389          0.00324   
39607 2021-12-31       0   3        0              0.0              0.0   
39608 2021-12-31       0   3        0         0.007885         0.005754   

      cctv_제한속도_평균_dong  cctv_count_dong  cctv_위도_std_gu  cctv_경도_std_gu  \
0                   8.0                5        0.004502        0.008730   
1                  17.5               12        0.018203        0.021047   
2                  55.0                4        0.017701        0.032648   
3                  38.0               10        0.021800        0.029514   
4                  40.0               12        0.036155        0.039577   
...                 ...              ...             ...             ...   
39604              40.0                1        0.017701        0.032648   
39605         26.666667               30        0.018203        0.021047   
39606         16.818182               22        0.018203        0.021047   
39607              60.0                1        0.018203        0.021047   
39608         47.857143               14        0.008486        0.011955   

       cctv_제한속도_평균_gu  cctv_count_gu park_위도_std_dong park_경도_std_dong  \
0            17.808219             73         0.003476          0.00259   
1            23.862816            277         0.003383         0.000738   
2            50.784314            102         0.000045         0.000063   
3            45.920000            125         0.002354         0.002494   
4            55.434783             92         0.000045         0.000063   
...                ...            ...              ...              ...   
39604        50.784314            102         0.000045         0.000063   
39605        23.862816            277         0.004368         0.008302   
39606        23.862816            277         0.000045         0.000063   
39607        23.862816            277         0.000045         0.000063   
39608        47.272727             55         0.000045         0.000063   

       주차구획수_dong park_위도_std_gu park_경도_std_gu  주차구획수_gu child_위도_std_dong  \
0             500       0.004584       0.007961      7592          0.003487   
1             114       0.015224       0.023717      4526               0.0   
2              10       0.014076        0.03586      2427           0.00571   
3             374        0.02184       0.023598 

In [ ]:
test

사고일시  요일  기상상태  도로형태  노면상태  사고유형  구    동     연   월   일  시간  \
0      2022-01-01 01   5     2     1     0     0  6   99  2022   1   1   1   
1      2022-01-01 01   5     2     6     0     0  6  168  2022   1   1   1   
2      2022-01-01 04   5     2     1     0     1  6  117  2022   1   1   4   
3      2022-01-01 04   5     2     6     0     1  6  126  2022   1   1   4   
4      2022-01-01 06   5     2     1     0     1  1    4  2022   1   1   6   
...              ...  ..   ...   ...   ...   ... ..  ...   ...  ..  ..  ..   
10958  2022-12-31 18   5     2     8     0     1  0   38  2022  12  31  18   
10959  2022-12-31 18   5     2     6     0     1  6  123  2022  12  31  18   
10960  2022-12-31 20   5     2     6     0     1  6  134  2022  12  31  20   
10961  2022-12-31 20   5     2     0     0     1  6   77  2022  12  31  20   
10962  2022-12-31 21   5     2     0     0     1  3  195  2022  12  31  21   

              일시  season  분기  holiday cctv_위도_std_dong cctv_경도_std_dong  \
0     2022-01-01       0   0        1         0.005473         0.003891   
1     2022-01-01       0   0        1         0.003632         0.006233   
2     2022-01-01       0   0        1              0.0              0.0   
3     2022-01-01       0   0        1          0.00303         0.004689   
4     2022-01-01       0   0        1         0.003379         0.004823   
...          ...     ...  ..      ...              ...              ...   
10958 2022-12-31       0   3        0          0.00829         0.010965   
10959 2022-12-31       0   3        0         0.002854         0.004911   
10960 2022-12-31       0   3        0              0.0              0.0   
10961 2022-12-31       0   3        0         0.002105         0.003478   
10962 2022-12-31       0   3        0         0.004158         0.002067   

      cctv_제한속도_평균_dong  cctv_count_dong  cctv_위도_std_gu  cctv_경도_std_gu  \
0                  52.5                4        0.017701        0.032648   
1             47.142857                7        0.017701        0.032648   
2                  40.0                1        0.017701        0.032648   
3             42.727273               11        0.017701        0.032648   
4                  17.5               12        0.018203        0.021047   
...                 ...              ...             ...             ...   
10958         20.483871               62        0.007748        0.014683   
10959              52.5                4        0.017701        0.032648   
10960               0.0                1        0.017701        0.032648   
10961         44.285714                7        0.017701        0.032648   
10962              42.5                4        0.036155        0.039577   

       cctv_제한속도_평균_gu  cctv_count_gu park_위도_std_dong park_경도_std_dong  \
0            50.784314            102         0.000045         0.000063   
1            50.784314            102         0.000106         0.005575   
2            50.784314            102         0.000045         0.000063   
3            50.784314            102         0.001476         0.003329   
4            23.862816            277         0.003383         0.000738   
...                ...            ...              ...              ...   
10958        18.936170             94         0.008884         0.010487   
10959        50.784314            102         0.000045         0.000063   
10960        50.784314            102         0.000045         0.000063   
10961        50.784314            102         0.000045         0.000063   
10962        55.434783             92         0.000045         0.000063   

       주차구획수_dong park_위도_std_gu park_경도_std_gu  주차구획수_gu child_위도_std_dong  \
0              10       0.014076        0.03586      2427          0.001747   
1             183       0.014076        0.03586      2427          0.005185   
2              10       0.014076        0.03586      2427               0.0   
3             153       0.014076        0.03586 

In [ ]:
data.drop(['사고일시','일시'], axis = 1, inplace = True)
test.drop(['사고일시','일시'], axis = 1, inplace = True)

# [4] 모델링
- idea

In [ ]:
!pip install scipy

In [ ]:
from pycaret.regression import *

In [ ]:
X = pd.concat([data, data_y], axis = 1)
X.head(2)

요일  기상상태  도로형태  노면상태  사고유형  구   동     연  월  일  시간  season  분기  holiday  \
0   6     2     6     0     0  7  40  2019  1  1   0       0   0        1   
1   6     5     6     0     0  1   4  2019  1  1   0       0   0        1   

  cctv_위도_std_dong cctv_경도_std_dong cctv_제한속도_평균_dong  cctv_count_dong  \
0         0.001808         0.001888               8.0                5   
1         0.003379         0.004823              17.5               12   

   cctv_위도_std_gu  cctv_경도_std_gu  cctv_제한속도_평균_gu  cctv_count_gu  \
0        0.004502        0.008730        17.808219             73   
1        0.018203        0.021047        23.862816            277   

  park_위도_std_dong park_경도_std_dong  주차구획수_dong park_위도_std_gu park_경도_std_gu  \
0         0.003476          0.00259         500       0.004584       0.007961   
1         0.003383         0.000738         114       0.015224       0.023717   

   주차구획수_gu child_위도_std_dong child_경도_std_dong  CCTV설치대수_dong  \
0      7592          0.003487          0.000775             13   
1      4526               0.0               0.0              0   

   child_count_dong child_위도_std_gu child_경도_std_gu  CCTV설치대수_gu  \
0                 2        0.004374        0.009301          148   
1                 1        0.004374        0.009301            0   

   child_count_gu  security_위도_std_dong  security_경도_std_dong  \
0              21              0.002577              0.002355   
1              21              0.003525              0.003298   

   security설치대수_dong  security_위도_std_gu  security_경도_std_gu  security설치대수_gu  \
0              391.0            0.005514            0.009685             5468   
1              932.0            0.016638            0.021791            10083   

   ECLO  
0     5  
1     3

In [ ]:
reg = setup(data = X # 데이터
      ,target = 'ECLO' # Target
      ,verbose = True # 진행 로그 출력
      ,train_size = 0.7 # 내부적으로 검증시 훈련데이터의 비율
      ,data_split_shuffle = True # 데이터 split시 데이터를 섞을것인지 여부
      ,normalize = True # 정규화 여부
      ,session_id = 22 # 난수 시드
      )

In [ ]:
models()

Name  \
ID                                            
lr                        Linear Regression   
lasso                      Lasso Regression   
ridge                      Ridge Regression   
en                              Elastic Net   
lar                  Least Angle Regression   
llar           Lasso Least Angle Regression   
omp             Orthogonal Matching Pursuit   
br                           Bayesian Ridge   
ard       Automatic Relevance Determination   
par            Passive Aggressive Regressor   
ransac              Random Sample Consensus   
tr                       TheilSen Regressor   
huber                       Huber Regressor   
kr                             Kernel Ridge   
svm               Support Vector Regression   
knn                   K Neighbors Regressor   
dt                  Decision Tree Regressor   
rf                  Random Forest Regressor   
et                    Extra Trees Regressor   
ada                      AdaBoost Regressor   
gbr             Gradient Boosting Regressor   
mlp                           MLP Regressor   
xgboost           Extreme Gradient Boosting   
lightgbm    Light Gradient Boosting Machine   
catboost                 CatBoost Regressor   
dummy                       Dummy Regressor   

                                                  Reference  Turbo  
ID                                                                  
lr              sklearn.linear_model._base.LinearRegression   True  
lasso        sklearn.linear_model._coordinate_descent.Lasso   True  
ridge                     sklearn.linear_model._ridge.Ridge   True  
en        sklearn.linear_model._coordinate_descent.Elast...   True  
lar                  sklearn.linear_model._least_angle.Lars   True  
llar            sklearn.linear_model._least_angle.LassoLars   True  
omp       sklearn.linear_model._omp.OrthogonalMatchingPu...   True  
br                sklearn.linear_model._bayes.BayesianRidge   True  
ard               sklearn.linear_model._bayes.ARDRegression  False  
par       sklearn.linear_model._passive_aggressive.Passi...   True  
ransac         sklearn.linear_model._ransac.RANSACRegressor  False  
tr        sklearn.linear_model._theil_sen.TheilSenRegressor  False  
huber            sklearn.linear_model._huber.HuberRegressor   True  
kr                         sklearn.kernel_ridge.KernelRidge  False  
svm                                sklearn.svm._classes.SVR  False  
knn       sklearn.neighbors._regression.KNeighborsRegressor   True  
dt              sklearn.tree._classes.DecisionTreeRegressor   True  
rf           sklearn.ensemble._forest.RandomForestRegressor   True  
et             sklearn.ensemble._forest.ExtraTreesRegressor   True  
ada       sklearn.ensemble._weight_boosting.AdaBoostRegr...   True  
gbr          sklearn.ensemble._gb.GradientBoostingRegressor   True  
mlp       sklearn.neural_network._multilayer_perceptron....  False  
xgboost                        xgboost.sklearn.XGBRegressor   True  
lightgbm                     lightgbm.sklearn.LGBMRegressor   True  
catboost                    catboost.core.CatBoostRegressor   True  
dummy                          sklearn.dummy.DummyRegressor   True

In [ ]:
top_7_model = compare_models(fold = 10, round = 5, n_select = 5, sort = 'rmsle')

Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [ ]:
blended = blend_models(estimator_list = top_7_model,
             fold = 10
             ,optimize = 'rmsle'
            )

Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
model = finalize_model(blended)
model

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['요일', '기상상태', '도로형태', '노면상태',
                                             '사고유형', '구', '동', '연', '월', '일',
                                             '시간', 'season', '분기', 'holiday',
                                             'cctv_count_dong',
                                             'cctv_위도_std_gu', 'cctv_경도_std_gu',
                                             'cctv_제한속도_평균_gu', 'cctv_count_gu',
                                             '주차구획수_dong', '주차구획수_gu',
                                             'CCTV설치대수_dong',
                                             'child_count_dong', 'CCTV설치대수_gu',
                                             'child_count_gu...
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('actual_estimator',
                 VotingRegressor(estimators=[('Huber Regressor',
                                              HuberRegressor()),
                                             ('Gradient Boosting Regressor',
                                              GradientBoostingRegressor(random_state=22)),
                                             ('Light Gradient Boosting Machine',
                                              LGBMRegressor(n_jobs=-1,
                                                            random_state=22)),
                                             ('Bayesian Ridge',
                                              BayesianRidge()),
                                             ('Ridge Regression',
                                              Ridge(random_state=22))],
                                 n_jobs=-1))])

In [ ]:
y_pred = predict_model(model, data = test)
y_pred.head(5)

요일  기상상태  도로형태  노면상태  사고유형  구    동     연  월  일  ...  child_경도_std_gu  \
0   5     2     1     0     0  6   99  2022  1  1  ...         0.035686   
1   5     2     6     0     0  6  168  2022  1  1  ...         0.035686   
2   5     2     1     0     1  6  117  2022  1  1  ...         0.035686   
3   5     2     6     0     1  6  126  2022  1  1  ...         0.035686   
4   5     2     1     0     1  1    4  2022  1  1  ...         0.009301   

   CCTV설치대수_gu  child_count_gu  security_위도_std_dong security_경도_std_dong  \
0            0             118              0.004207             0.005234   
1            0             118              0.004207             0.005234   
2            0             118              0.004207             0.005234   
3            0             118              0.004207             0.005234   
4            0              21              0.003525             0.003298   

  security설치대수_dong security_위도_std_gu  security_경도_std_gu  security설치대수_gu  \
0        700.000000           0.019177            0.023295            11911   
1        850.816223           0.019177            0.023295            11911   
2        850.816223           0.019177            0.023295            11911   
3        850.816223           0.019177            0.023295            11911   
4        932.000000           0.016638            0.021791            10083   

   prediction_label  
0          4.641217  
1          4.135784  
2          5.481495  
3          5.003578  
4          4.985356  

[5 rows x 43 columns]

In [ ]:
sub['ECLO'] = y_pred.prediction_label

In [ ]:
sub

ID      ECLO
0      ACCIDENT_39609  4.641217
1      ACCIDENT_39610  4.135784
2      ACCIDENT_39611  5.481495
3      ACCIDENT_39612  5.003578
4      ACCIDENT_39613  4.985356
...               ...       ...
10958  ACCIDENT_50567  4.813531
10959  ACCIDENT_50568  4.564523
10960  ACCIDENT_50569  4.495626
10961  ACCIDENT_50570  4.761037
10962  ACCIDENT_50571  4.863421

[10963 rows x 2 columns]

In [ ]:
sub.to_csv('/content/drive/MyDrive/Dacon/23_대구교통사고피해예측AI경진대회/submissions/submission_4.csv', index = False)